# Coins Kata

This kata consists in implement the code for a coin changer calculator, so given an amount, and a set of coins, it will return the quantity of each coin required to give that amount.

The exercise is thought to be completed in three iterations.

## Basic coin changer

In this iteration, the coin changer will receive an amount to provide with a predefined set of coins.
This set of coins contains six following classes:

* 5 cents
* 10 cents
* 20 cents
* 50 cents
* 1 euro
* 2 euros


For this iteration, the following rules are to be followed:

* The amount will always be multiple of 5 cents, as a float number with two decimal positions.
* The system will have coins enough of each class.
* There's no limit on the quantity and class of coins used.

For instance, let's say the amount is 25 cents, so the program could work like this:

    python coins.py 0.25
    5 x 5c
    
Other possible amounts would be like the following:

    python coins.py 2.35
    47 x 5c
    python coins.py 3.00
    60 x 5c
    python coins.py 5.50
    110 x 5c

This iteration result is intended to be simple and neat, without too much complications.
The expected time to complete this iteration is 15 to 20 minutes.

In [1]:
%%writefile tests.py
import unittest
import coins

class TestCoinChanger(unittest.TestCase):
    def test_5c(self):
        pass

if __name__ == '__main__':
    unittest.main()

Overwriting tests.py


In [2]:
%%writefile coins.py
def change(amount):
    return {'5c': 0}

Overwriting coins.py


In [3]:
%%bash
python tests.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [4]:
%%writefile tests.py
import unittest
import coins

class TestCoinChanger(unittest.TestCase):
    def test_5c(self):
        money = coins.change(0.05)
        self.assertEqual(money['5c'], 1)

if __name__ == '__main__':
    unittest.main()

Overwriting tests.py


In [5]:
%%bash
python tests.py

F
FAIL: test_5c (__main__.TestCoinChanger)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests.py", line 7, in test_5c
    self.assertEqual(money['5c'], 1)
AssertionError: 0 != 1

----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (failures=1)


In [6]:
%%writefile coins.py
def change(amount):
    return {'5c': 1}

Overwriting coins.py


In [7]:
%%bash
python tests.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [8]:
%%writefile tests.py
import unittest
import coins

def coin_value(coin):
    if coin.rfind('c') > 0:
        return float(coin.strip('c')) / 100
    else:
        return float(coin.strip('e'))

class TestCoinChanger(unittest.TestCase):
    def test_5c(self):
        money = coins.change(0.05)
        self.assertEqual(money['5c'], 1)

    def test_generic(self):
        amounts = [0.05, 0.10]
        for amount in amounts:
            money = coins.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))

if __name__ == '__main__':
    unittest.main()

Overwriting tests.py


In [9]:
%%bash
python tests.py

.F
FAIL: test_generic (__main__.TestCoinChanger)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests.py", line 20, in test_generic
    self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
AssertionError: 0.05 != 0.1 : Change value 0.05 differs from the amount 0.05

----------------------------------------------------------------------
Ran 2 tests in 0.000s

FAILED (failures=1)


In [10]:
%%writefile coins.py
def change(amount):
    return {'5c': (amount * 100) // 5}

Overwriting coins.py


In [11]:
%%bash
python tests.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


This `tests.py` checks everything works with more amounts:

In [12]:
%%writefile tests.py
import unittest
import coins

def coin_value(coin):
    if coin.rfind('c') > 0:
        return float(coin.strip('c')) / 100
    else:
        return float(coin.strip('e'))

class TestCoinChanger(unittest.TestCase):
    def test_5c(self):
        money = coins.change(0.05)
        self.assertEqual(money['5c'], 1)

    def test_generic(self):
        amounts = [0.05, 0.10, 0.25, 2.35, 3.00, 5.50]
        for amount in amounts:
            money = coins.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))

if __name__ == '__main__':
    unittest.main()

Overwriting tests.py


In [13]:
%%bash
python tests.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## Optimal coin changer

In this iteration, the result should be the optimal quantity of coins, so the classes should be used in such a way, that the total number of coins is the lesser possible.

For the same amounts of the previous iteration examples, we would have:

    python coins.py 0.25
    1 x 20c, 1 x 5c, total 2 coins
    python coins.py 2.35
    1 x 2e, 1 x 20c, 1 x 10c, 1 x 5c, total 4 coins
    python coins.py 3.00
    1 x 2e, 1 x 1e, total 2 coins
    python coins.py 5.50
    2 x 2e, 1 x 1e, 1 x 50c total 4 coins

This iteration shouldn't take more than 30-45 minutes.

In [14]:
%%writefile coins.py
coins = ['5c', '10c', '20c', '50c', '1e', '2e']

def change(amount):
    return {'5c': (amount * 100) // 5}

Overwriting coins.py


In [15]:
%%writefile tests.py
import unittest
import coins

def coin_value(coin):
    if coin.rfind('c') > 0:
        return float(coin.strip('c')) / 100
    else:
        return float(coin.strip('e'))

def quantity_is_optimal(money):
    """
    This function iterates over all possible coins checking
    if the amount for this coin and quantity used might be
    the same with a higher value coin.
    """
    accum = 0.0
    for i, k in enumerate(coins.coins):
        if k in money.keys():
            accum += money[k] * coin_value(k)
            if i + 1 < len(coins.coins):
                n = coins.coins[i + 1]
                if n in money.keys() and money[n] > 0:
                    next_value = money[n] * coin_value(n)
                else:
                    next_value = 1 * coin_value(n)
            else:
                return True
            if accum >= next_value:
                return False
    return True

class TestCoinChanger(unittest.TestCase):
    def test_5c(self):
        money = coins.change(0.05)
        self.assertEqual(money['5c'], 1)

    def test_generic(self):
        amounts = [0.05, 0.10, 0.25, 2.35, 3.00, 5.50]
        for amount in amounts:
            money = coins.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
            self.assertTrue(quantity_is_optimal(money), "Coin types used are not optimal for {0:.2f}".format(amount))

if __name__ == '__main__':
    unittest.main()

Overwriting tests.py


In [16]:
%%bash
python tests.py

.F
FAIL: test_generic (__main__.TestCoinChanger)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests.py", line 43, in test_generic
    self.assertTrue(quantity_is_optimal(money), "Coin types used are not optimal for {0:.2f}".format(amount))
AssertionError: False is not true : Coin types used are not optimal for 0.10

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=1)


In [17]:
%%writefile coins.py
coins = ['5c', '10c', '20c', '50c', '1e', '2e']
denominations = {'5c': 0.05, '10c': 0.1, '20c': 0.2, '50c': 0.5, '1e': 1, '2e': 2}
reversed_coins = coins.copy()
reversed_coins.reverse()

def change(amount):
    result = {}
    for coin in reversed_coins:
        result[coin] = amount // denominations[coin]
        amount -= result[coin] * denominations[coin]
        amount = round(amount, 2)
    return result

Overwriting coins.py


In [18]:
%%bash
python tests.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## Optimal limited coin changer

For this iteration, also the quantity of coins of each class is limited, so the program will be limited to have the following quantities:

* 5 cents: 2 coins
* 10 cents: 2 coins
* 20 cents: 2 coins
* 50 cents: 2 coins
* 1 euro: 2 coins
* 2 euro: 1 coin

For this iteration, there will be no amount which will require more coins than these.

For the examples followed so far, the results should be:

    python coins.py 0.25
    1 x 20c, 1 x 5c, total 2 coins
    python coins.py 2.35
    1 x 2e, 1 x 20c, 1 x 10c, 1 x 5c, total 4 coins
    python coins.py 3.00
    1 x 2e, 1 x 1e, total 2 coins
    python coins.py 5.50
    1 x 2e, 2 x 1e, 2 x 50c, 2 x 20c, 1 x 10c, total 8 coins

In [19]:
%%writefile tests.py
import unittest
import coins

def coin_value(coin):
    if coin.rfind('c') > 0:
        return float(coin.strip('c')) / 100
    else:
        return float(coin.strip('e'))

def quantity_is_optimal(money, wallet = None):
    """
    This function iterates over all possible coins checking
    if the amount for this coin and quantity used might be
    the same with a higher value coin.
    """
    accum = 0.0
    for i, k in enumerate(coins.coins):
        if k in money.keys():
            accum += money[k] * coin_value(k)
            if i + 1 < len(coins.coins):
                n = coins.coins[i + 1]
                if n in money.keys() and money[n] > 0:
                    next_value = money[n] * coin_value(n)
                else:
                    next_value = 1 * coin_value(n)
            else:
                return True
            if wallet is None and accum >= next_value:
                return False
    return True

def extract_list(change):
    coins = ['5c', '10c', '20c', '50c', '1e', '2e']
    base = []
    for k in coins:
        base.append(change[k])
    return base
    
def consumed_too_much(wallet, change):   
    return False in [False for n, m in zip(extract_list(wallet), extract_list(change)) if n - m < 0]
        
class TestCoinChanger(unittest.TestCase):
    def test_5c(self):
        money = coins.change(0.05)
        self.assertEqual(money['5c'], 1)

    def test_generic(self):
        amounts = [0.05, 0.10, 0.25, 2.35, 3.00, 5.50]
        for amount in amounts:
            money = coins.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
            self.assertTrue(quantity_is_optimal(money), "Coin types used are not optimal for {0:.2f}".format(amount))

    def test_limited(self):
        amounts = [0.05, 0.10, 0.25, 2.35, 3.00, 5.50]
        for amount in amounts:
            wallet = {'5c': 2, '10c': 2, '20c': 2, '50c': 2, '1e': 2, '2e': 1}
            money = coins.change(amount, wallet)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
            self.assertFalse(consumed_too_much(wallet, money), "Wallet exhausted for {0:.2f}".format(amount))
            self.assertTrue(quantity_is_optimal(money, wallet), "Coin types used are not optimal for {0:.2f}".format(amount))

if __name__ == '__main__':
    unittest.main()

Overwriting tests.py


In [20]:
%%writefile coins.py
coins = ['5c', '10c', '20c', '50c', '1e', '2e']
denominations = {'5c': 0.05, '10c': 0.1, '20c': 0.2, '50c': 0.5, '1e': 1, '2e': 2}
reversed_coins = coins.copy()
reversed_coins.reverse()

def change(amount, wallet=None):
    result = {}
    for coin in reversed_coins:
        result[coin] = amount // denominations[coin]
        amount -= result[coin] * denominations[coin]
        amount = round(amount, 2)
    return result

Overwriting coins.py


In [21]:
%%bash
python tests.py

..F
FAIL: test_limited (__main__.TestCoinChanger)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests.py", line 62, in test_limited
    self.assertFalse(consumed_too_much(wallet, money), "Wallet exhausted for {0:.2f}".format(amount))
AssertionError: True is not false : Wallet exhausted for 5.50

----------------------------------------------------------------------
Ran 3 tests in 0.001s

FAILED (failures=1)


In [22]:
%%writefile coins.py
coins = ['5c', '10c', '20c', '50c', '1e', '2e']
denominations = {'5c': 0.05, '10c': 0.1, '20c': 0.2, '50c': 0.5, '1e': 1, '2e': 2}
reversed_coins = coins.copy()
reversed_coins.reverse()

def change(amount, wallet = None):
    original = amount
    diff = 1
    if wallet is None:
        wallet = {'2e': 1, '1e': 1, '50c': 1, '20c': 1, '10c': 1, '5c': 1}
        diff = 0
    result = {}
    for coin in reversed_coins:
        result[coin] = amount // denominations[coin]
        if wallet[coin] < result[coin]:
            carry = (result[coin] - wallet[coin]) * diff
            result[coin] -= carry
        amount -= result[coin] * denominations[coin]
        amount = round(amount, 2)
    return result

Overwriting coins.py


In [23]:
%%bash
python tests.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


## Currency defined optimal limited changer

In [24]:
%%writefile tests.py
import unittest
import coins

def coin_value(coin):
    if coin.rfind('c') > 0:
        return float(coin.strip('c')) / 100
    elif coin.rfind('e') > 0:
        return float(coin.strip('e'))
    elif coin.rfind('d') > 0:
        return float(coin.strip('d'))

def quantity_is_optimal(money, coins, wallet = None):
    """
    This function iterates over all possible coins checking
    if the amount for this coin and quantity used might be
    the same with a higher value coin.
    """
    accum = 0.0
    for i, k in enumerate(coins):
        if k in money.keys():
            accum += money[k] * coin_value(k)
            if i + 1 < len(coins):
                n = coins[i + 1]
                if n in money.keys() and money[n] > 0:
                    next_value = money[n] * coin_value(n)
                else:
                    next_value = 1 * coin_value(n)
            else:
                return True
            if wallet is None and accum >= next_value:
                return False
    return True

def extract_list(change, coins):
    base = []
    for k in coins:
        base.append(change[k])
    return base
    
def consumed_too_much(wallet, change, coins):   
    return False in [False for n, m in zip(extract_list(wallet, coins), extract_list(change, coins)) if n - m < 0]
        
class TestCoinChanger(unittest.TestCase):
    def setUp(self):
        self.euroChanger = coins.CoinChanger()
        self.euroWallet = {'5c': 2, '10c': 2, '20c': 2, '50c': 2, '1e': 2, '2e': 1}
        self.euroChangerWithWallet = coins.CoinChanger(self.euroWallet)

    def test_5c(self):
        money = self.euroChanger.change(0.05)
        self.assertEqual(money['5c'], 1)

    def test_generic(self):
        amounts = [0.05, 0.10, 0.25, 2.35, 3.00, 5.50]
        for amount in amounts:
            money = self.euroChanger.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
            self.assertTrue(quantity_is_optimal(money, coins.coins), "Coin types used are not optimal for {0:.2f}".format(amount))

    def test_limited(self):
        amounts = [0.05, 0.10, 0.25, 2.35, 3.00, 5.50]
        for amount in amounts:
            money = self.euroChangerWithWallet.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
            self.assertFalse(consumed_too_much(self.euroWallet, money, coins.coins), "Wallet exhausted for {0:.2f}".format(amount))
            self.assertTrue(quantity_is_optimal(money, coins.coins, self.euroWallet), "Coin types used are not optimal for {0:.2f}".format(amount))

if __name__ == '__main__':
    unittest.main()

Overwriting tests.py


In [25]:
%%writefile coins.py
coins = ['5c', '10c', '20c', '50c', '1e', '2e']
denominations = {'5c': 0.05, '10c': 0.1, '20c': 0.2, '50c': 0.5, '1e': 1, '2e': 2}
reversed_coins = coins.copy()
reversed_coins.reverse()

def change(amount, wallet = None):
    original = amount
    diff = 1
    if wallet is None:
        wallet = {'2e': 1, '1e': 1, '50c': 1, '20c': 1, '10c': 1, '5c': 1}
        diff = 0
    result = {}
    for coin in reversed_coins:
        result[coin] = amount // denominations[coin]
        if wallet[coin] < result[coin]:
            carry = (result[coin] - wallet[coin]) * diff
            result[coin] -= carry
        amount -= result[coin] * denominations[coin]
        amount = round(amount, 2)
    return result


class CoinChanger(object):
    def __init__(self, wallet=None):
        self.coins = coins
        self.denominations = denominations
        self.wallet = wallet

    def change(self, amount):
        return {'5c': 0}

Overwriting coins.py


In [26]:
%%bash
python tests.py

FFF
FAIL: test_5c (__main__.TestCoinChanger)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests.py", line 51, in test_5c
    self.assertEqual(money['5c'], 1)
AssertionError: 0 != 1

FAIL: test_generic (__main__.TestCoinChanger)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests.py", line 58, in test_generic
    self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
AssertionError: 0.0 != 0.05 : Change value 0.00 differs from the amount 0.00

FAIL: test_limited (__main__.TestCoinChanger)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests.py", line 66, in test_limited
    self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
AssertionError: 0.0 != 0.05 : Change value 0.00

In [27]:
%%writefile coins.py
coins = ['5c', '10c', '20c', '50c', '1e', '2e']
denominations = {'5c': 0.05, '10c': 0.1, '20c': 0.2, '50c': 0.5, '1e': 1, '2e': 2}
reversed_coins = coins.copy()
reversed_coins.reverse()

class CoinChanger(object):
    def __init__(self, wallet=None):
        if wallet is None:
            self.wallet = {'2e': 1, '1e': 1, '50c': 1, '20c': 1, '10c': 1, '5c': 1}
            self.diff = 0
        else:
            self.wallet = wallet
            self.diff = 1

    def change(self, amount):
        original = amount
        result = {}
        for coin in reversed_coins:
            result[coin] = amount // denominations[coin]
            if self.wallet[coin] < result[coin]:
                carry = (result[coin] - self.wallet[coin]) * self.diff
                result[coin] -= carry
            amount -= result[coin] * denominations[coin]
            amount = round(amount, 2)
        return result

Overwriting coins.py


In [28]:
%%bash
python tests.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


Now, refactor to add new currencies:

In [29]:
%%writefile tests.py
import unittest
import coins

def coin_value(coin):
    if coin.rfind('c') > 0:
        return float(coin.strip('c')) / 100
    elif coin.rfind('e') > 0:
        return float(coin.strip('e'))
    elif coin.rfind('d') > 0:
        return float(coin.strip('d'))

def quantity_is_optimal(money, coins, wallet = None):
    """
    This function iterates over all possible coins checking
    if the amount for this coin and quantity used might be
    the same with a higher value coin.
    """
    accum = 0.0
    for i, k in enumerate(coins):
        if k in money.keys():
            accum += money[k] * coin_value(k)
            if i + 1 < len(coins):
                n = coins[i + 1]
                if n in money.keys() and money[n] > 0:
                    next_value = money[n] * coin_value(n)
                else:
                    next_value = 1 * coin_value(n)
            else:
                return True
            if wallet is None and accum >= next_value:
                return False
    return True

def extract_list(change, coins):
    base = []
    for k in coins:
        base.append(change[k])
    return base
    
def consumed_too_much(wallet, change, coins):   
    return False in [False for n, m in zip(extract_list(wallet, coins), extract_list(change, coins)) if n - m < 0]
        
class TestCoinChanger(unittest.TestCase):
    def setUp(self):
        self.euroChanger = coins.CoinChanger()
        self.euroWallet = {'5c': 2, '10c': 2, '20c': 2, '50c': 2, '1e': 2, '2e': 1}
        self.euroChangerWithWallet = coins.CoinChanger(wallet=self.euroWallet)
        self.usChanger = coins.CoinChanger(coins.US_COINS, coins.US_DENOMS)
        self.usWallet = {'5c': 2, '10c': 2, '25c': 2, '50c': 4, '1d': 3}
        self.usChangerWithWallet = coins.CoinChanger(coins.US_COINS, coins.US_DENOMS,wallet=self.usWallet)

    def test_5c(self):
        money = self.euroChanger.change(0.05)
        self.assertEqual(money['5c'], 1)
        money = self.usChanger.change(0.05)
        self.assertEqual(money['5c'], 1)

    def test_generic(self):
        amounts = [0.05, 0.10, 0.25, 2.35, 3.00, 5.50]
        for amount in amounts:
            money = self.euroChanger.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
            self.assertTrue(quantity_is_optimal(money, coins.EURO_COINS), "Coin types used are not optimal for {0:.2f}".format(amount))
        for amount in amounts:
            money = self.usChanger.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
            self.assertTrue(quantity_is_optimal(money, coins.US_COINS), "Coin types used are not optimal for {0:.2f}".format(amount))

    def test_limited(self):
        amounts = [0.05, 0.10, 0.25, 2.35, 3.00, 5.50]
        for amount in amounts:
            money = self.euroChangerWithWallet.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
            self.assertFalse(consumed_too_much(self.euroWallet, money, coins.EURO_COINS), "Wallet exhausted for {0:.2f}".format(amount))
            self.assertTrue(quantity_is_optimal(money, coins.EURO_COINS, self.euroWallet), "Coin types used are not optimal for {0:.2f}".format(amount))
        for amount in amounts:
            money = self.usChangerWithWallet.change(amount)
            change = round(sum([q * coin_value(k) for k, q in money.items()]), 2)
            self.assertEqual(change, amount, "Change value {0:.2f} differs from the amount {0:.2f}".format(change, amount))
            self.assertFalse(consumed_too_much(self.usWallet, money, coins.US_COINS), "Wallet exhausted for {0:.2f}".format(amount))
            self.assertTrue(quantity_is_optimal(money, coins.US_COINS, self.usWallet), "Coin types used are not optimal for {0:.2f}".format(amount))

if __name__ == '__main__':
    unittest.main()

Overwriting tests.py


In [30]:
%%writefile coins.py
US_COINS = ['5c', '10c', '25c', '50c', '1d']
EURO_COINS = ['5c', '10c', '20c', '50c', '1e', '2e']
US_DENOMS = {'5c': 0.05, '10c': 0.1, '25c': 0.25, '50c': 0.5, '1d': 1}
EURO_DENOMS = {'5c': 0.05, '10c': 0.1, '20c': 0.2, '50c': 0.5, '1e': 1, '2e': 2}
US_WALLET = {'1d': 1, '50c': 1, '25c': 1, '10c': 1, '5c': 1}
EURO_WALLET = {'2e': 1, '1e': 1, '50c': 1, '20c': 1, '10c': 1, '5c': 1}

class CoinChanger(object):
    def __init__(self, coins=None, denoms=None, wallet=None):
        self.coins = coins
        if coins is None:
            self.coins = EURO_COINS
        self.denoms = denoms
        if denoms is None:
            self.denoms = EURO_DENOMS
        self.reversed_coins = self.coins.copy()
        self.reversed_coins.reverse()
        if wallet is None:
            self.wallet = EURO_WALLET
            if '1d' in self.coins:
                self.wallet = US_WALLET
            self.diff = 0
        else:
            self.wallet = wallet
            self.diff = 1

    def change(self, amount):
        original = amount
        result = {}
        for coin in self.reversed_coins:
            result[coin] = amount // self.denoms[coin]
            if self.wallet[coin] < result[coin]:
                carry = (result[coin] - self.wallet[coin]) * self.diff
                result[coin] -= carry
            amount -= result[coin] * self.denoms[coin]
            amount = round(amount, 2)
        return result

Overwriting coins.py


In [31]:
%%bash
python tests.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
